In [2]:
import pandas as pd
cust_names = ['a', 'b', 'c']
cust_info = [1.2, 2.3, 3.4]
cust_series = pd.Series(cust_info, cust_names)
print(list(cust_series[cust_series > 1.5].index))

['b', 'c']


In [3]:
cust_series[['a', 'c']]

a    1.2
c    3.4
dtype: float64

In [4]:
cust_series.loc[['a', 'c']]

a    1.2
c    3.4
dtype: float64

In [5]:
cust_series.iloc[[0, 2]]

a    1.2
c    3.4
dtype: float64

# Case Study: From Data to Diagnosis: Handling Missing Values and Aggregating Patient Records with Pandas

### Business Problem
### A hospital wants to improve patient care and operational efficiency.
### They need to analyze patient encounter data to:

### ** Identify trends in readmissions and costs

### ** Filter and segment patients by encounter type and payer

### ** Handle missing or incomplete records

### ** Aggregate costs and stay durations by patient groups

### ** Sort and merge information for actionable insights

#### 1: Load Data

In [6]:
# Loading Dataset
df = pd.read_csv('encounters.csv')
df.head()

,Id,START,STOP,PATIENT,ORGANIZATION,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,32c84703-2481-49cd-d571-3899d5820253,2011-01-02T09:26:36Z,2011-01-02T12:58:36Z,3de74169-7f67-9304-91d4-757e0f3a14d2,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,185347001,Encounter for problem (procedure),85.55,1018.02,0.00,NaN,NaN
1,c98059da-320a-c0a6-fced-c8815f3e3f39,2011-01-03T05:44:39Z,2011-01-03T06:01:42Z,d9ec2e44-32e9-9148-179a-1653348cc4e2,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,outpatient,308335008,Patient encounter procedure,142.58,2619.36,0.00,NaN,NaN
2,4ad28a3a-2479-782b-f29c-d5b3f41a001e,2011-01-03T14:32:11Z,2011-01-03T14:47:11Z,73babadf-5b2b-fee7-189e-6f41ff213e01,d78e84ec-30aa-3bba-a33a-f29a3a454662,7caa7254-5050-3b5e-9eae-bd5ea30e809c,outpatient,185349003,Encounter for check up (procedure),85.55,461.59,305.27,NaN,NaN
3,c3f4da61-e4b4-21d5-587a-fbc89943bc19,2011-01-03T16:24:45Z,2011-01-03T16:39:45Z,3b46a0b7-0f34-9b9a-c319-ace4a1f58c0b,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,wellness,162673000,General examination of patient (procedure),136.80,1784.24,0.00,NaN,NaN
4,a9183b4f-2572-72ea-54c2-b3cd038b4be7,2011-01-03T17:36:53Z,2011-01-03T17:51:53Z,fa006887-d93c-d302-8b89-f3c25f88c0e1,d78e84ec-30aa-3bba-a33a-f29a3a454662,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,ambulatory,390906007,Follow-up encounter,85.55,234.72,0.00,55822004.0,Hyperlipidemia


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27891 entries, 0 to 27890
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   27891 non-null  object 
 1   START                27891 non-null  object 
 2   STOP                 27891 non-null  object 
 3   PATIENT              27891 non-null  object 
 4   ORGANIZATION         27891 non-null  object 
 5   PAYER                27891 non-null  object 
 6   ENCOUNTERCLASS       27891 non-null  object 
 7   CODE                 27891 non-null  int64  
 8   DESCRIPTION          27891 non-null  object 
 9   BASE_ENCOUNTER_COST  27891 non-null  float64
 10  TOTAL_CLAIM_COST     27891 non-null  float64
 11  PAYER_COVERAGE       27891 non-null  float64
 12  REASONCODE           8350 non-null   float64
 13  REASONDESCRIPTION    8350 non-null   object 
dtypes: float64(4), int64(1), object(9)
memory usage: 3.0+ MB


#### Column Selection and Filtering

In [9]:
columns_int = ['Id', 'PATIENT', 'ENCOUNTERCLASS', 'BASE_ENCOUNTER_COST', 'PAYER', 'REASONDESCRIPTION']
columns_int = [c.lower() for c in columns_int]

In [12]:
selected_columns = [col for col in df.columns if col.lower() in columns_int]
df[selected_columns]

,Id,PATIENT,PAYER,ENCOUNTERCLASS,BASE_ENCOUNTER_COST,REASONDESCRIPTION
0,32c84703-2481-49cd-d571-3899d5820253,3de74169-7f67-9304-91d4-757e0f3a14d2,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,85.55,NaN
1,c98059da-320a-c0a6-fced-c8815f3e3f39,d9ec2e44-32e9-9148-179a-1653348cc4e2,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,outpatient,142.58,NaN
2,4ad28a3a-2479-782b-f29c-d5b3f41a001e,73babadf-5b2b-fee7-189e-6f41ff213e01,7caa7254-5050-3b5e-9eae-bd5ea30e809c,outpatient,85.55,NaN
3,c3f4da61-e4b4-21d5-587a-fbc89943bc19,3b46a0b7-0f34-9b9a-c319-ace4a1f58c0b,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,wellness,136.80,NaN
4,a9183b4f-2572-72ea-54c2-b3cd038b4be7,fa006887-d93c-d302-8b89-f3c25f88c0e1,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,ambulatory,85.55,Hyperlipidemia
...,...,...,...,...,...,...
27886,917bb534-2ed3-e1f5-ac27-74f314d19c63,0c122290-8c15-77ac-b418-2fea4e8a8e6a,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,outpatient,85.55,NaN
27887,07710480-9d6b-9c9b-87c3-c1d54df4069d,20a4bc24-6b69-2f5c-dc74-1df390cae25b,7caa7254-5050-3b5e-9eae-bd5ea30e809c,urgentcare,142.58,NaN
27888,01b57f06-cebe-a3e4-4423-a796ffb0c35d,ff1b3c26-53a6-4590-ce79-a3f7269274ea,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,142.58,Normal pregnancy
27889,7d435668-0813-eb3b-0f26-cb741fb39561,ff1b3c26-53a6-4590-ce79-a3f7269274ea,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,wellness,136.80,NaN


#### 3:  Handlind Missing Values

In [13]:
df.isnull().sum()

Id                         0
START                      0
STOP                       0
PATIENT                    0
ORGANIZATION               0
PAYER                      0
ENCOUNTERCLASS             0
CODE                       0
DESCRIPTION                0
BASE_ENCOUNTER_COST        0
TOTAL_CLAIM_COST           0
PAYER_COVERAGE             0
REASONCODE             19541
REASONDESCRIPTION      19541
dtype: int64

In [14]:
# Fill Misssing BASE_ENCOUNTER_COST With Median Value
df["BASE_ENCOUNTER_COST"].fillna(df["BASE_ENCOUNTER_COST"].median(), inplace=True)

C:\Users\kumar\AppData\Local\Temp\ipykernel_22072\1247807837.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["BASE_ENCOUNTER_COST"].fillna(df["BASE_ENCOUNTER_COST"].median(), inplace=True)


In [15]:
# Drop rows where Reason Description is Missing
df.dropna(subset=["REASONDESCRIPTION"], inplace=True)
df.isnull().sum()

Id                     0
START                  0
STOP                   0
PATIENT                0
ORGANIZATION           0
PAYER                  0
ENCOUNTERCLASS         0
CODE                   0
DESCRIPTION            0
BASE_ENCOUNTER_COST    0
TOTAL_CLAIM_COST       0
PAYER_COVERAGE         0
REASONCODE             0
REASONDESCRIPTION      0
dtype: int64

#### 4: Aggregate costs and stay durations by patient groups

In [17]:
# Average of Base Encounter cost per encounter class
df.groupby('ENCOUNTERCLASS')["BASE_ENCOUNTER_COST"].mean()

ENCOUNTERCLASS
ambulatory    110.034084
emergency     139.201323
inpatient     135.808129
outpatient     85.550000
wellness      136.800000
Name: BASE_ENCOUNTER_COST, dtype: float64

In [18]:
#Grouping for total claim by payer
df.groupby('PAYER')["TOTAL_CLAIM_COST"].sum()

PAYER
047f6ec3-6215-35eb-9608-f9dda363a44c    1195513.62
42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a    1700220.24
4d71f845-a6a9-3c39-b242-14d25ef86a8d    1091943.15
5059a55e-5d6e-34d1-b6cb-d83d16e57bcf    1051841.75
6e2f1a2d-27bd-3701-8d08-dae202c58632    1623290.20
7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a    7165889.64
7caa7254-5050-3b5e-9eae-bd5ea30e809c    2828182.83
b1c428d6-4f07-31e0-90f0-68ffa6ff8c76    9983932.33
b3221cfc-24fb-339e-823d-bc4136cbc4ed     534523.99
d47b3510-2895-3b70-9897-342d681c769d    1354417.07
Name: TOTAL_CLAIM_COST, dtype: float64

#### 5 : Sorting

In [22]:
# Sort by Highest base cost
sorted_df = df.sort_values(by="BASE_ENCOUNTER_COST", ascending=False)
sorted_df.head()

,Id,START,STOP,PATIENT,ORGANIZATION,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
9282,90e4e925-7e6f-2be4-1936-ebbdd0b4f72c,2014-10-23T21:29:00Z,2014-10-24T21:29:00Z,cd2f81e1-ac45-c9af-6a3a-b526e22194c7,d78e84ec-30aa-3bba-a33a-f29a3a454662,7caa7254-5050-3b5e-9eae-bd5ea30e809c,inpatient,310061009,Gynecology service (qualifier value),146.18,2101.07,1648.86,254837009.0,Malignant neoplasm of breast (disorder)
24285,bb3c86bd-82f2-5649-1143-1ee3b6212de6,2021-01-19T21:18:02Z,2021-01-28T05:46:41Z,9b90e343-201b-b66c-15ed-31e4f0f18030,d78e84ec-30aa-3bba-a33a-f29a3a454662,7caa7254-5050-3b5e-9eae-bd5ea30e809c,inpatient,305351004,Admission to intensive care unit (procedure),146.18,16990.14,13550.22,840539006.0,COVID-19
24198,c5ee7ce0-7cb1-35eb-8997-c01f6974dc46,2021-01-10T07:03:12Z,2021-01-10T08:03:12Z,7015216e-a987-963d-2dd3-afae2d18cb15,d78e84ec-30aa-3bba-a33a-f29a3a454662,047f6ec3-6215-35eb-9608-f9dda363a44c,emergency,183460006,Obstetric emergency hospital admission,146.18,4333.97,64.64,72892002.0,Normal pregnancy
24215,142a6a81-5fe2-c090-1161-ea59a6ccefc8,2021-01-11T20:37:54Z,2021-01-19T21:18:02Z,9b90e343-201b-b66c-15ed-31e4f0f18030,d78e84ec-30aa-3bba-a33a-f29a3a454662,7caa7254-5050-3b5e-9eae-bd5ea30e809c,inpatient,1505002,Hospital admission for isolation (procedure),146.18,19702.88,15682.99,840539006.0,COVID-19
5164,4e1da21d-49ec-5426-e752-b920e74b255a,2013-09-04T02:44:17Z,2013-09-04T03:44:17Z,2dae7142-8892-cc88-cf5f-24868b0fbd75,d78e84ec-30aa-3bba-a33a-f29a3a454662,047f6ec3-6215-35eb-9608-f9dda363a44c,emergency,50849002,Emergency Room Admission,146.18,146.18,0.00,55680006.0,Drug overdose


## Challenge: 

- Filter all Emergency encounters , fill using claim costs with the mean, and find the average claim cost for emergencies

In [24]:
df_emergency = df[df['ENCOUNTERCLASS'] == "emergency"].copy()

In [25]:
mean_claim = df_emergency["TOTAL_CLAIM_COST"].mean()
df_emergency.loc[: , "TOTAL_CLAIM_COST"] = df_emergency.loc[: , "TOTAL_CLAIM_COST"].fillna(mean_claim)

In [29]:
round(df_emergency["TOTAL_CLAIM_COST"].mean(),2)

5193.45